<a href="https://colab.research.google.com/github/Weirdo-001/FundRaise/blob/main/70.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
SCRIPTS_ZIP_PATH = "/content/drive/MyDrive/Hackathon2_scripts.zip"
DATASET_ZIP_PATH = "/content/drive/MyDrive/datasets.zip"

!mkdir -p /content/hackathon_scripts
!mkdir -p /content/datasets

!cp "{SCRIPTS_ZIP_PATH}" /content/
!cp "{DATASET_ZIP_PATH}" /content/

!unzip -qo /content/Hackathon2_scripts.zip -d /content/hackathon_scripts
!unzip -qo /content/datasets.zip -d /content/datasets

!ls /content/hackathon_scripts
!ls /content/datasets
!ls /content/datasets/datasets/train/images | head -n 5


Hackathon2_scripts
datasets
000000000_cluttered_hallway.png
000000000_cluttered_room.png
000000000_light_cluttered_room.png
000000001_light_cluttered.png
000000001_light_cluttered_room.png


In [3]:
!pip install -q ultralytics==8.2.10 matplotlib seaborn opencv-python tqdm pyyaml
!pip install -q -U ultralytics

import ultralytics
ultralytics.checks()

Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 68.8/112.6 GB disk)


In [4]:
data_yaml_content = {
    'train': '/content/datasets/datasets/train/images',
    'val': '/content/datasets/datasets/val/images',
    'test': '/content/datasets/datasets/test/images',
    'nc': 7,
    'names': ['OxygenTank', 'NitrogenTank', 'FirstAidBox', 'FireAlarm',
              'SafetySwitchPanel', 'EmergencyPhone', 'FireExtinguisher']
}




In [5]:
import yaml
data_yaml_path = "/content/datasets/data.yaml"
with open(data_yaml_path, "w") as f:
    yaml.dump(data_yaml_content, f)
print("✅ data.yaml created at", data_yaml_path)

✅ data.yaml created at /content/datasets/data.yaml


In [6]:
!rm -f yolov8m.pt
!curl -L https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8m.pt -o yolov8m.pt
print("✅ Pretrained YOLOv8m weights downloaded")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 49.7M  100 49.7M    0     0  36.5M      0  0:00:01  0:00:01 --:--:-- 60.9M
✅ Pretrained YOLOv8m weights downloaded


In [7]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")

In [8]:
EPOCHS = 30                  # More epochs for better learning
BATCH = 16                    # Reduce if GPU OOM occurs
IMGSZ = 640
MOSAIC = 0.7
MIXUP = 0.2
LR0 = 0.0003
LRF = 0.01

# 🔹 7️⃣ Advanced augmentation & safe training config
results = model.train(
    data=data_yaml_path,
    model="yolov8m.pt",
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    augment=True,         # basic augmentation
    mosaic=MOSAIC,        # mosaic augmentation
    mixup=MIXUP,          # mixup augmentation
    lr0=LR0,
    lrf=LRF,
    patience=10,          # early stopping
    flipud=0.5,           # vertical flip
    fliplr=0.5,           # horizontal flip
    hsv_h=0.015,          # HSV augmentation
    hsv_s=0.7,
    hsv_v=0.4,
    box=7.5,              # default box loss gain
    device=0,             # GPU
    save_period=1,        # Save checkpoints each epoch
    cache=True,           # Cache images for speed (if memory allows)
    workers=4,            # parallel data loading
)

print("✅ Training completed!")


Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.7, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, 

In [ ]:
metrics = model.val(data_yaml_path, batch=BATCH, imgsz=IMGSZ)
print("✅ Validation metrics:", metrics)

# 🔹 9️⃣ Visualize predictions (sample 5 images from test)
import cv2, matplotlib.pyplot as plt
test_images = sorted(os.listdir(data_yaml_content['test']))[:5]

for img_name in test_images:
    img_path = os.path.join(data_yaml_content['test'], img_name)
    results = model.predict(img_path, imgsz=IMGSZ, conf=0.25, save=False)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    annotated_img = results[0].plot()
    plt.figure(figsize=(8,6))
    plt.imshow(annotated_img)
    plt.title(img_name)
    plt.axis('off')
    plt.show()